In [2]:
pip install pyserial


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 3.0 MB/s eta 0:00:00


In [3]:
import serial
import time
import random
import numpy as np


In [12]:
import serial
import time
import random
import numpy as np


CROP_PROFILES = {
    "rice": {
        "cold_limit": 15, "optimal_min": 20, "optimal_max": 30, "hot_limit": 35,
    },
    "wheat": {
        "cold_limit": 10, "optimal_min": 15, "optimal_max": 25, "hot_limit": 30,
    },
    "corn": {
        "cold_limit": 18, "optimal_min": 22, "optimal_max": 32, "hot_limit": 38,
    },
    "pulses": {
        "cold_limit": 12, "optimal_min": 18, "optimal_max": 28, "hot_limit": 35,
    },
}


def is_cold(temp, crop_profile):
    cold_limit, optimal_min = crop_profile["cold_limit"], crop_profile["optimal_min"]
    if temp < cold_limit: return 1.0
    if cold_limit <= temp <= optimal_min: return (optimal_min - temp) / (optimal_min - cold_limit)
    return 0.0

def is_optimal(temp, crop_profile):
    optimal_min, optimal_max = crop_profile["optimal_min"], crop_profile["optimal_max"]
    hot_limit, cold_limit = crop_profile["hot_limit"], crop_profile["cold_limit"]
    if optimal_min <= temp <= optimal_max: return 1.0
    if cold_limit < temp < optimal_min: return (temp - cold_limit) / (optimal_min - cold_limit)
    if optimal_max < temp < hot_limit: return (hot_limit - temp) / (hot_limit - optimal_max)
    return 0.0

def is_hot(temp, crop_profile):
    hot_limit, optimal_max = crop_profile["hot_limit"], crop_profile["optimal_max"]
    if temp > hot_limit: return 1.0
    if optimal_max <= temp <= hot_limit: return (temp - optimal_max) / (hot_limit - optimal_max)
    return 0.0


def evaluate_temp_stress(temp, crop_name):
    if crop_name not in CROP_PROFILES:
        return {"status": "ERROR: Crop profile not found.", "confidence": 0.0}
    crop_profile = CROP_PROFILES[crop_name]
    cold_level = is_cold(temp, crop_profile)
    optimal_level = is_optimal(temp, crop_profile)
    hot_level = is_hot(temp, crop_profile)
    if cold_level > 0.8 or hot_level > 0.8:
        return {"status": "CRITICAL: High risk of crop damage!", "confidence": max(cold_level, hot_level)}
    if cold_level > 0.3 or hot_level > 0.3:
        return {"status": "WARNING: Temperature is a potential risk.", "confidence": max(cold_level, hot_level)}
    if optimal_level > 0.5:
        return {"status": "Optimal: Crop health is stable.", "confidence": optimal_level}
    return {"status": "Neutral: Status uncertain.", "confidence": 0.0}


if __name__ == "__main__":
    serial_port = "COM3"
    baud_rate = 9600

    try:
        arduino = serial.Serial(serial_port, baud_rate, timeout=1)
        time.sleep(2)
        print(f"Connected to Arduino on port {serial_port}")
    except serial.SerialException as e:
        print(f"Error connecting to Arduino: {e}")
        print("Falling back to simulated data. Please check your port and connection.")
        arduino = None

    target_crop = "corn"
    print(f"Monitoring Temperature for: {target_crop.capitalize()}")
    print("-" * 70)
    print(f"{'Temperature (°C)':<20} | {'Status':<40} | {'Confidence':<15}")
    print("=" * 70)


    while True:
        try:
            current_temp = None
            if arduino:
                line = arduino.readline().decode('utf-8').strip()
                if line and "temp" in line:
                    parts = line.split(',')
                    data = {}
                    for part in parts:
                        if ':' in part:
                            key, value = part.split(':')
                            data[key.strip()] = float(value.strip())
                    current_temp = data.get("temp")
            else:

                current_temp = random.uniform(5, 45)

            if current_temp is not None:
                temp_status = evaluate_temp_stress(current_temp, target_crop)
                print(f"{current_temp:<20.2f} | {temp_status['status']:<40} | {temp_status['confidence']:.2f}")

            time.sleep(2)

        except (serial.SerialException, ValueError) as e:
            print(f"\nAn error occurred: {e}")
            print("Trying to reconnect or continuing with simulation...\n")
            if arduino:
                try:
                    arduino.close()
                    arduino = serial.Serial(serial_port, baud_rate, timeout=1)
                    time.sleep(2)
                    print("Reconnected to Arduino.")
                except serial.SerialException:
                    arduino = None
                    print("Failed to reconnect. Using simulated data.")
            time.sleep(5)
            continue


Error connecting to Arduino: [Errno 2] could not open port COM3: [Errno 2] No such file or directory: 'COM3'
Falling back to simulated data. Please check your port and connection.
Monitoring Temperature for: Corn
----------------------------------------------------------------------
Temperature (°C)     | Status                                   | Confidence     
10.77                | CRITICAL: High risk of crop damage!      | 1.00
29.89                | Optimal: Crop health is stable.          | 1.00


KeyboardInterrupt: 

In [11]:
import serial
import time
import random

CROP_PROFILES = {
    "rice": {
        "moisture_min": 50, "moisture_max": 90,
    },
    "wheat": {
        "moisture_min": 30, "moisture_max": 65,
    },
    "corn": {
        "moisture_min": 40, "moisture_max": 75,
    },
    "pulses": {
        "moisture_min": 25, "moisture_max": 60,
    },
}

def moisture_is_low(moisture, profile):
    min_val = profile["moisture_min"]
    if moisture < min_val: return 1.0
    if min_val <= moisture <= min_val + 10: return (min_val + 10 - moisture) / 10
    return 0.0

def moisture_is_optimal(moisture, profile):
    min_val, max_val = profile["moisture_min"], profile["moisture_max"]
    if min_val <= moisture <= max_val: return 1.0
    if min_val - 10 < moisture < min_val: return (moisture - (min_val - 10)) / 10
    if max_val < moisture < max_val + 10: return (max_val + 10 - moisture) / 10
    return 0.0

def moisture_is_high(moisture, profile):
    max_val = profile["moisture_max"]
    if moisture > max_val + 10: return 1.0
    if max_val <= moisture <= max_val + 10: return (moisture - max_val) / 10
    return 0.0

def evaluate_moisture_stress(moisture, crop_name):
    if crop_name not in CROP_PROFILES:
        return {"status": "ERROR: Crop profile not found.", "confidence": 0.0}
    profile = CROP_PROFILES[crop_name]
    low_level = moisture_is_low(moisture, profile)
    optimal_level = moisture_is_optimal(moisture, profile)
    high_level = moisture_is_high(moisture, profile)
    if low_level > 0.8:
        return {"status": "CRITICAL: Low soil moisture. Risk of drought stress!", "confidence": low_level}
    if high_level > 0.8:
        return {"status": "CRITICAL: High soil moisture. Risk of root rot!", "confidence": high_level}
    if optimal_level > 0.5:
        return {"status": "Optimal: Soil moisture is stable.", "confidence": optimal_level}
    return {"status": "Neutral: Moisture status uncertain.", "confidence": 0.0}

if __name__ == "__main__":
    serial_port = "COM3"
    baud_rate = 9600

    try:
        arduino = serial.Serial(serial_port, baud_rate, timeout=1)
        time.sleep(2)
        print(f"Connected to Arduino on port {serial_port}")
    except serial.SerialException as e:
        print(f"Error connecting to Arduino: {e}")
        print("Falling back to simulated data. Please check your port and connection.")
        arduino = None

    target_crop = "corn"
    print(f"Monitoring Soil Moisture for: {target_crop.capitalize()}")
    print("-" * 80)
    print(f"{'Soil Moisture (%)':<20} | {'Status':<50} | {'Confidence':<15}")
    print("=" * 80)


    while True:
        try:
            current_moisture = None
            if arduino:
                line = arduino.readline().decode('utf-8').strip()
                if line and "moisture" in line:
                    parts = line.split(',')
                    data = {}
                    for part in parts:
                        if ':' in part:
                            key, value = part.split(':')
                            data[key.strip()] = float(value.strip())
                    current_moisture = data.get("moisture")
            else:

                current_moisture = random.uniform(10, 95)

            if current_moisture is not None:
                moisture_status = evaluate_moisture_stress(current_moisture, target_crop)
                print(f"{current_moisture:<20.2f} | {moisture_status['status']:<50} | {moisture_status['confidence']:.2f}")

            time.sleep(2)

        except (serial.SerialException, ValueError) as e:
            print(f"\nAn error occurred: {e}")
            print("Trying to reconnect or continuing with simulation...\n")
            if arduino:
                try:
                    arduino.close()
                    arduino = serial.Serial(serial_port, baud_rate, timeout=1)
                    time.sleep(2)
                    print("Reconnected to Arduino.")
                except serial.SerialException:
                    arduino = None
                    print("Failed to reconnect. Using simulated data.")
            time.sleep(5)
            continue


Error connecting to Arduino: [Errno 2] could not open port COM3: [Errno 2] No such file or directory: 'COM3'
Falling back to simulated data. Please check your port and connection.
Monitoring Soil Moisture for: Corn
--------------------------------------------------------------------------------
Soil Moisture (%)    | Status                                             | Confidence     
94.54                | CRITICAL: High soil moisture. Risk of root rot!    | 1.00
69.08                | Optimal: Soil moisture is stable.                  | 1.00
31.76                | CRITICAL: Low soil moisture. Risk of drought stress! | 1.00
75.46                | Optimal: Soil moisture is stable.                  | 0.95
26.90                | CRITICAL: Low soil moisture. Risk of drought stress! | 1.00
53.47                | Optimal: Soil moisture is stable.                  | 1.00
63.15                | Optimal: Soil moisture is stable.                  | 1.00
41.39                | CRITICAL: Low soil

KeyboardInterrupt: 

In [7]:
import serial
import time
import random

CROP_PROFILES = {
    "rice": {
        "humidity_min": 60, "humidity_max": 85,
    },
    "wheat": {
        "humidity_min": 40, "humidity_max": 60,
    },
    "corn": {
        "humidity_min": 50, "humidity_max": 70,
    },
    "pulses": {
        "humidity_min": 35, "humidity_max": 55,
    },
}

def humidity_is_low(humidity, profile):
    min_val = profile["humidity_min"]
    if humidity < min_val - 10: return 1.0
    if min_val - 10 <= humidity <= min_val: return (min_val - humidity) / 10
    return 0.0

def humidity_is_optimal(humidity, profile):
    min_val, max_val = profile["humidity_min"], profile["humidity_max"]
    if min_val <= humidity <= max_val: return 1.0
    if min_val - 10 < humidity < min_val: return (humidity - (min_val - 10)) / 10
    if max_val < humidity < max_val + 10: return (max_val + 10 - humidity) / 10
    return 0.0

def humidity_is_high(humidity, profile):
    max_val = profile["humidity_max"]
    if humidity > max_val + 10: return 1.0
    if max_val <= humidity <= max_val + 10: return (humidity - max_val) / 10
    return 0.0

def evaluate_humidity_stress(humidity, crop_name):
    if crop_name not in CROP_PROFILES:
        return {"status": "ERROR: Crop profile not found.", "confidence": 0.0}
    profile = CROP_PROFILES[crop_name]
    low_level = humidity_is_low(humidity, profile)
    optimal_level = humidity_is_optimal(humidity, profile)
    high_level = humidity_is_high(humidity, profile)
    if high_level > 0.8:
        return {"status": "CRITICAL: High humidity. Fungal disease risk!", "confidence": high_level}
    if low_level > 0.8:
        return {"status": "CRITICAL: Low humidity. Plant dehydration risk!", "confidence": low_level}
    if optimal_level > 0.5:
        return {"status": "Optimal: Air humidity is stable.", "confidence": optimal_level}
    return {"status": "Neutral: Humidity status uncertain.", "confidence": 0.0}

if __name__ == "__main__":
    serial_port = "COM3"
    baud_rate = 9600

    try:
        arduino = serial.Serial(serial_port, baud_rate, timeout=1)
        time.sleep(2)
        print(f"Connected to Arduino on port {serial_port}")
    except serial.SerialException as e:
        print(f"Error connecting to Arduino: {e}")
        print("Falling back to simulated data. Please check your port and connection.")
        arduino = None

    target_crop = "pulses"
    print(f"Monitoring Air Humidity for: {target_crop.capitalize()}")
    print("-" * 80)
    print(f"{'Air Humidity (%)':<20} | {'Status':<50} | {'Confidence':<15}")
    print("=" * 80)

    while True:
        try:
            current_humidity = None
            if arduino:
                line = arduino.readline().decode('utf-8').strip()
                if line and "humidity" in line:
                    parts = line.split(',')
                    data = {}
                    for part in parts:
                        if ':' in part:
                            key, value = part.split(':')
                            data[key.strip()] = float(value.strip())
                    current_humidity = data.get("humidity")
            else:

                current_humidity = random.uniform(30, 99)

            if current_humidity is not None:
                humidity_status = evaluate_humidity_stress(current_humidity, target_crop)
                print(f"{current_humidity:<20.2f} | {humidity_status['status']:<50} | {humidity_status['confidence']:.2f}")

            time.sleep(2)

        except (serial.SerialException, ValueError) as e:
            print(f"\nAn error occurred: {e}")
            print("Trying to reconnect or continuing with simulation...\n")
            if arduino:
                try:
                    arduino.close()
                    arduino = serial.Serial(serial_port, baud_rate, timeout=1)
                    time.sleep(2)
                    print("Reconnected to Arduino.")
                except serial.SerialException:
                    arduino = None
                    print("Failed to reconnect. Using simulated data.")
            time.sleep(5)
            continue


Error connecting to Arduino: [Errno 2] could not open port COM3: [Errno 2] No such file or directory: 'COM3'
Falling back to simulated data. Please check your port and connection.
Monitoring Air Humidity for: Pulses
--------------------------------------------------------------------------------
Air Humidity (%)     | Status                                             | Confidence     
83.76                | CRITICAL: High humidity. Fungal disease risk!      | 1.00
32.08                | Optimal: Air humidity is stable.                   | 0.71
50.90                | Optimal: Air humidity is stable.                   | 1.00
98.58                | CRITICAL: High humidity. Fungal disease risk!      | 1.00
83.72                | CRITICAL: High humidity. Fungal disease risk!      | 1.00
33.90                | Optimal: Air humidity is stable.                   | 0.89
86.84                | CRITICAL: High humidity. Fungal disease risk!      | 1.00
65.53                | CRITICAL: High humidi

KeyboardInterrupt: 